<a href="https://colab.research.google.com/github/HakkiAkut/ai-ml-algorithms/blob/master/LSTM/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content/drive/MyDrive/ML/lstm/

/content/drive/MyDrive/ML/lstm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.losses import binary_crossentropy

the dataset is from https://www.kaggle.com/team-ai/spam-text-message-classification

In [ ]:
dataset = pd.read_csv("dataset.csv", encoding='latin-1') # receive data from dataset.csv
dataset = dataset.sample(frac=1)  # shuffle
dataset.shape

(5572, 2)

Splitting dataset to labels and features

In [ ]:
def data_split(data):
  labelEncoder= LabelEncoder()
  labels= data.iloc[0:,0].values # takes labels which is 1th column for dataset.csv 
  labels= labelEncoder.fit_transform(labels)
  features = data.iloc[0:,1].values # takes features which is 2th column for dataset.csv 
  return features, labels

In [ ]:
features, labels= data_split(dataset)

Finding total number of words and max length of a message

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(features)
features_seq = tokenizer.texts_to_sequences(features)
max_length = max([len(item) for item in features_seq])
word_size = len(tokenizer.word_index) + 1
word_size, max_length

(9008, 189)

Adding shape to data since I'm using keras sequential, its needed to have a shape.

if word_size and  max_size stays 9008 and 189 respectively, the metrics are val_loss: 0.1687 - val_accuracy: 0.8543

I decided to limit word_size and max_size to 1500 and 100 and the metrics becomes val_loss: 0.0583 - val_accuracy: 0.9821 (way better)

In [ ]:
word_size, max_length = 1500, 100
tokenizer = Tokenizer(num_words=word_size)
tokenizer.fit_on_texts(features)
features = pad_sequences(tokenizer.texts_to_sequences(features), maxlen = max_length) 

dividing dataset to train and test

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size =0.2, random_state=42) # split into training set and test set
print(train_x.shape,test_x.shape) # train size, test size

(4457, 100) (1115, 100)


Initializing the model

In [ ]:
model= Sequential()

Adding layers to model

In [ ]:
model.add(Embedding(word_size, 64, input_length=max_length, name='Embedding1')) # need to add Embedding layer since working with text

model.add(LSTM(64,return_sequences=True, activation='relu', name= 'LSTM1'))
model.add(Dropout(0.2))

model.add(LSTM(128,activation='relu', name='LSTM2'))
model.add(Dropout(0.2)) 

model.add(Dense(128,activation='relu', name='Dense1'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu', name='Dense2'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu', name='Dense3'))
model.add(Dropout(0.2))
model.add(Dense(16,activation='relu', name='Dense4'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid', name='Output'))

In [ ]:
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])

Summary

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding1 (Embedding)      (None, 100, 64)           96000     
                                                                 
 LSTM1 (LSTM)                (None, 100, 64)           33024     
                                                                 
 dropout_66 (Dropout)        (None, 100, 64)           0         
                                                                 
 LSTM2 (LSTM)                (None, 128)               98816     
                                                                 
 dropout_67 (Dropout)        (None, 128)               0         
                                                                 
 Dense1 (Dense)              (None, 128)               16512     
                                                                 
 dropout_68 (Dropout)        (None, 128)             

Training

In [ ]:
model.fit(train_x, train_y, batch_size=128, epochs=10, validation_data=(test_x, test_y))

Epoch 1/10
35/35 [==============================] - 19s 470ms/step - loss: 1213.2220 - accuracy: 0.8474 - val_loss: 0.5658 - val_accuracy: 0.8619
Epoch 2/10
35/35 [==============================] - 16s 456ms/step - loss: 0.5388 - accuracy: 0.8643 - val_loss: 0.4867 - val_accuracy: 0.8619
Epoch 3/10
35/35 [==============================] - 16s 457ms/step - loss: 0.4405 - accuracy: 0.8609 - val_loss: 0.3250 - val_accuracy: 0.8619
Epoch 4/10
35/35 [==============================] - 16s 456ms/step - loss: 0.3053 - accuracy: 0.8636 - val_loss: 0.1916 - val_accuracy: 0.8619
Epoch 5/10
35/35 [==============================] - 16s 458ms/step - loss: 0.3632 - accuracy: 0.8782 - val_loss: 0.1339 - val_accuracy: 0.9561
Epoch 6/10
35/35 [==============================] - 16s 457ms/step - loss: 0.3476 - accuracy: 0.9670 - val_loss: 0.0716 - val_accuracy: 0.9830
Epoch 7/10
35/35 [==============================] - 16s 457ms/step - loss: 0.0756 - accuracy: 0.9805 - val_loss: 0.0634 - val_accuracy: 0.9

The Metrics

In [ ]:
loss, accuracy = model.evaluate(test_x, test_y, batch_size=128)
print(f'loss: {loss}, accuracy: {accuracy}')

9/9 [==============================] - 1s 138ms/step - loss: 0.0583 - accuracy: 0.9821
loss: 0.05830492824316025, accuracy: 0.9820627570152283


I tried to do spam ham text classification, the metrics are: loss is 0.0583, and accuracy 0.9821. Which is a pretty good model.